In [2]:
#import matplotlib as mpl
#from mpl_toolkits.mplot3d import Axes3D
import numpy as np
#import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#import quadpy
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations
from joblib import Parallel, delayed, parallel_backend
from scipy import integrate
import time
from datetime import timedelta
import plotly as plotly
import plotly.express as px
#from plotly.offline import init_notebook_mode
#init_notebook_mode(connected=True)  
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.graph_objects as go

import os

#if not os.path.exists("/mnt/home/dmassa/FCC_supercells/RDF/RDFimages"):
#    os.mkdir("/mnt/home/dmassa/FCC_supercells/RDF/RDFimages")
    
def crystal(i):
    switcher={
        45: "Li", 
        44: "Cu",
        43: "Mn",
        42: "Al",            
        41: "Au",            
        40: "Ca",      
        39: "Cr",        
        38: "Fe",            
        37: "Ge",            
        36: "Ir",      
        35: "Pd",        
        34: "Pt",            
        33: "Ru",            
        32: "Si",      
        31: "Tc",
        30: "Ti",
        29: "V",
        28: "W",
        27: "Mg",
        26: "Co",
        25: "Sc", 
        24: "Hg",
        23: "Mo", 
        22: "Os",
        21: "Rh",
        20: "Ag",
        19: "Cd",
        18: "Hf",
        17: "K",
        16: "Na",
        15: "Nb",
        14: "Ni",
        13: "Re",
        12: "Sb",
        11: "Sr",
        10: "Ta",
        9: "Y",
        8: "Zr",
        7: "Ba",
        6: "Cs",
        5: "In",
        4: "Rb",
        3: "Sn",
        2: "Pb",
        1: "Tl",
    }
    return switcher.get(i,"Invalid crystal")

suffix="TETRA"

df = pd.read_csv("Final_Differences_Only_"+suffix+".csv")

def f(nx0, ny0, nz0, pp, n_H):
    
    #print(n_H[pp-1])
    
    
    if (nx0 <= int(101-n_H[pp-1])):
        nx0 = int( nx0 + n_H[pp-1] -1) 
    else: 
        nx0 = int( nx0 + n_H[pp-1] -101 -1)

    if (ny0 <= int(101-n_H[pp-1])):
        ny0 = int( ny0 + n_H[pp-1] -1) 
    else: 
        ny0 = int( ny0 + n_H[pp-1] -101 -1)
        
    if (nz0 <= int(101-n_H[pp-1])):
        nz0 = int( nz0 + n_H[pp-1] -1) 
    else: 
        nz0 = int( nz0 + n_H[pp-1] -101 -1)
    
    crystalname=crystal(pp)
    
   # print("RUNNING:",crystalname)
        
    charge = df[crystalname+"_Difference"]
    
    charge=np.array(charge)
    
    charge=charge.reshape((dimension,dimension,dimension))

    nx = np.arange(charge.shape[0])#[:, None, None]
    ny = np.arange(charge.shape[1])#[None, :, None]
    nz = np.arange(charge.shape[2])#[None, None, :]
    
    nx, ny, nz = np.broadcast_arrays(nx, ny, nz)

    return charge[nx0, ny0, nz0]


def process(f,pp,n_H,nx0,ny0,nz0):
    
    # print ('nx=',nx0)
    # res=integrate.nquad(f,[[nx0,nx0+1], [ny0,ny0+1], [nz0,nz0+1]],args=(pp,n_H),full_output=False)
    res = f(nx0, ny0, nz0, pp, n_H)
   # time.sleep(0.03)
    
    #print(res[0])
    return res


def plotter(res,R_cont,crystalname):

    R_cont=np.array(R_cont)

    R=R_cont.astype(int)

    Rr=np.unique(R_cont.ravel()) 

    Rrmax=max(Rr)

    tbin = np.bincount(R, weights=res)
    
    nr = np.bincount(R)

    RDF = tbin/nr
    
    Rr_axis=np.linspace(0,Rrmax,len(nr)) #== Rr?

    charge_DF={"R":Rr_axis,
              "RDF":RDF}

    # fig = px.scatter(charge_DF,x="R", y="RDF", color="RDF")

    # plotly.offline.plot(fig, filename='RDFimages_notint/RDF_'+crystalname+'_'+suffix+'.html')
    
    return RDF
    

    
    
#------------

R_cont=[]
for nx in range (0,101):
    for ny in range(0,101):
        for nz in range(0,101):
            
            R_cont.append(np.sqrt(nx**2+ny**2+nz**2))


dimension=101


df_intcharge = pd.DataFrame(list())

n_H = np.zeros(45)
res=[]
#------------

if suffix == "OCTA": 
    mult_fact=4.0  # true for 2x2x2
else: 
    mult_fact=8.0 # true for 2x2x2

for pp in range (1,46):
    
    if pp==7 or pp==26 or pp==38 or pp==14 or pp==28 or pp==18 or pp==24:
        continue
        
    if pp==6 or pp==32 or pp==12:
        continue
    
    nx_level = list(range(0,101))
    ny_level = list(range(0,101))
    nz_level = list(range(0,101))

    crystalname=crystal(pp)

    #charge = df_tetra[crystalname+"_Difference"]

    with open('../H_positions/H_position_grid_'+suffix+'_'+crystalname+'.dat') as file:
        for line in file:
            data = line.split()
            L_H=data[0]
            L_H=float(L_H)
            L=L_H*mult_fact
    
    deltaN = L/101.0

    n_H [pp-1] = round(L_H/deltaN)

    start_time = time.monotonic()   

    print("Histogramming over:",crystalname)

    # for nx0 in nx_level:
        
    #     for ny0 in ny_level: 
            
    #         for nz0 in nz_level:
                
    #             res.append(process(f,pp,n_H,nx0,ny0,nz0))

    res.append(Parallel(n_jobs=6)(delayed(process)(f,pp,n_H,nx0,ny0,nz0) for nx0 in nx_level for ny0 in ny_level for nz0 in nz_level))

    res=np.asarray(res)

    res=np.reshape(res, res.size)
    
    end_time = time.monotonic()
    print(timedelta(seconds=end_time - start_time))

    # df_intcharge[crystalname+"_"+suffix+"_NotInt_Charge"] = pd.Series(res)
    # df_intcharge.to_csv("NotIntCharge_"+suffix+".csv", index=False) 

    RDF=plotter(res,R_cont,crystalname)
    RDF=np.asarray(RDF)
    df_intcharge[crystalname+"_"+suffix+"_RDF"] = RDF
    df_intcharge.to_csv("NotIntCharge_"+suffix+".csv", index=False) 
    


Histogramming over: Tl
0:05:43.236320
Histogramming over: Pb
0:05:32.216816
Histogramming over: Sn
0:05:35.352192
Histogramming over: Rb
0:05:17.028613
Histogramming over: In
0:05:00.612417
Histogramming over: Zr
0:04:41.018273
Histogramming over: Y
0:04:42.608584
Histogramming over: Ta
0:04:50.204608
Histogramming over: Sr
0:04:41.520070
Histogramming over: Re
0:04:41.283904
Histogramming over: Nb
0:04:34.179872
Histogramming over: Na
0:04:53.195390
Histogramming over: K
0:04:37.950496
Histogramming over: Cd
0:04:50.984675
Histogramming over: Ag
0:04:52.420982
Histogramming over: Rh
0:04:59.619575
Histogramming over: Os
0:04:31.275823
Histogramming over: Mo
0:04:45.164520
Histogramming over: Sc
0:05:21.538117
Histogramming over: Mg
0:04:55.994369
Histogramming over: V
0:04:44.220968
Histogramming over: Ti
0:04:58.710947
Histogramming over: Tc
0:04:54.261770
Histogramming over: Ru
0:05:00.516890
Histogramming over: Pt
0:04:47.604674
Histogramming over: Pd
0:05:00.459716
Histogramming o